In [1]:
import pandas as pd
import os
os.listdir()
os.listdir("Data/")#Contents of the Data folder

['category', 'product_views_unified.csv']

In [2]:
dataFile = "DATA/product_views_unified.csv"
header_list = ['user_id_anon','product_id']
empData = pd.read_csv(dataFile,usecols=header_list, nrows = 4000000)
empData.head(10)
data_2020 = empData #uz odfiltrovano
print(data_2020)

         user_id_anon    product_id
0            14723304  100011803451
1            14723304  100011612041
2            14723304  100016624290
3            14723304       1093107
4            14723304  100016287573
...               ...           ...
3999995      25330075       1340114
3999996      25330075       1303863
3999997      25330075       1309008
3999998      25330075       1309008
3999999      25330075        945053

[4000000 rows x 2 columns]


In [15]:
ids = data_2020['user_id_anon'] #filtruje users kteri se divali jen na jednu vec:
data_2020[ids.isin(ids[ids.duplicated()])].sort_values(['user_id_anon','date']) 

KeyError: 'date'

In [3]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [4]:
data_2020.dtypes

user_id_anon    int64
product_id      int64
dtype: object

In [5]:
data_2020 = data_2020.dropna().drop_duplicates()
print(len(data_2020))
data_2020 = data_2020[['user_id_anon','product_id']]

2893505


In [8]:
# data_2020.head()
# print(data_2020[data_2020['user_id_anon'] == 889])

In [6]:
#Preprocessing dat (ve formatu list stringů)
data_2020['product_id'] = data_2020['product_id'].map(str) 
sentences = data_2020.groupby('user_id_anon')['product_id'].apply(list)
print(sentences)

user_id_anon
645         [655028, 1338684, 100018643178, 100018644575, ...
788         [100016425290, 452568001, 1086821, 1403084, 14...
871         [551806, 1305893, 1293084, 1389660, 1510064, 1...
1345        [100018205007, 100018206462, 100018206445, 100...
2373        [100004358593, 100004358588, 100004360094, 100...
                                  ...                        
62563934    [100016141442, 100016143151, 100016145005, 100...
62565140    [1236035, 100002172473, 645720, 100002786928, ...
62565992    [1339130, 1330960002, 100020754345, 1302780, 1...
62568860    [100019452001, 1144838, 783693, 956190, 125758...
62570516    [1327813005, 673184004, 1455333, 1326999, 8158...
Name: product_id, Length: 83945, dtype: object


In [9]:
import multiprocessing

import gensim

from gensim.models import Word2Vec

cores = multiprocessing.cpu_count() # Count the number of cores in a computer

w2v_model = Word2Vec(min_count=2,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [10]:
t = time()

w2v_model.build_vocab(sentences, progress_per=5000) #vlozi slova do vocabulary

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 23:36:45: collecting all words and their counts
INFO - 23:36:45: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 23:36:45: PROGRESS: at sentence #5000, processed 174993 words, keeping 90085 word types
INFO - 23:36:45: PROGRESS: at sentence #10000, processed 346133 words, keeping 138473 word types
INFO - 23:36:45: PROGRESS: at sentence #15000, processed 518362 words, keeping 175134 word types
INFO - 23:36:45: PROGRESS: at sentence #20000, processed 689594 words, keeping 204776 word types
INFO - 23:36:45: PROGRESS: at sentence #25000, processed 866212 words, keeping 230770 word types
INFO - 23:36:45: PROGRESS: at sentence #30000, processed 1039599 words, keeping 253701 word types
INFO - 23:36:45: PROGRESS: at sentence #35000, processed 1211624 words, keeping 273345 word types
INFO - 23:36:45: PROGRESS: at sentence #40000, processed 1385766 words, keeping 291462 word types
INFO - 23:36:45: PROGRESS: at sentence #45000, processed 1557771 words, keeping 30790

Time to build vocab: 0.63 mins


In [11]:
t = time()

#trenovani modelu:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 23:37:29: training model with 15 workers on 234616 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2
INFO - 23:37:30: EPOCH 1 - PROGRESS: at 18.90% examples, 489327 words/s, in_qsize 25, out_qsize 4
INFO - 23:37:31: EPOCH 1 - PROGRESS: at 45.88% examples, 597945 words/s, in_qsize 26, out_qsize 3
INFO - 23:37:32: EPOCH 1 - PROGRESS: at 71.62% examples, 622018 words/s, in_qsize 30, out_qsize 1
INFO - 23:37:33: worker thread finished; awaiting finish of 14 more threads
INFO - 23:37:33: worker thread finished; awaiting finish of 13 more threads
INFO - 23:37:33: worker thread finished; awaiting finish of 12 more threads
INFO - 23:37:33: worker thread finished; awaiting finish of 11 more threads
INFO - 23:37:33: worker thread finished; awaiting finish of 10 more threads
INFO - 23:37:33: worker thread finished; awaiting finish of 9 more threads
INFO - 23:37:33: EPOCH 1 - PROGRESS: at 97.45% examples, 640144 words/s, in_qsize 8, out_qsize 1
INFO - 23:37:33: 

INFO - 23:37:51: EPOCH 6 - PROGRESS: at 22.44% examples, 605587 words/s, in_qsize 29, out_qsize 0
INFO - 23:37:52: EPOCH 6 - PROGRESS: at 46.20% examples, 628340 words/s, in_qsize 28, out_qsize 1
INFO - 23:37:53: EPOCH 6 - PROGRESS: at 71.63% examples, 640505 words/s, in_qsize 30, out_qsize 1
INFO - 23:37:54: worker thread finished; awaiting finish of 14 more threads
INFO - 23:37:54: worker thread finished; awaiting finish of 13 more threads
INFO - 23:37:54: worker thread finished; awaiting finish of 12 more threads
INFO - 23:37:54: EPOCH 6 - PROGRESS: at 96.39% examples, 647475 words/s, in_qsize 11, out_qsize 1
INFO - 23:37:54: worker thread finished; awaiting finish of 11 more threads
INFO - 23:37:54: worker thread finished; awaiting finish of 10 more threads
INFO - 23:37:54: worker thread finished; awaiting finish of 9 more threads
INFO - 23:37:54: worker thread finished; awaiting finish of 8 more threads
INFO - 23:37:54: worker thread finished; awaiting finish of 7 more threads
INF

INFO - 23:38:13: EPOCH 11 - PROGRESS: at 48.93% examples, 634086 words/s, in_qsize 29, out_qsize 0
INFO - 23:38:14: EPOCH 11 - PROGRESS: at 75.02% examples, 651258 words/s, in_qsize 30, out_qsize 2
INFO - 23:38:14: worker thread finished; awaiting finish of 14 more threads
INFO - 23:38:14: worker thread finished; awaiting finish of 13 more threads
INFO - 23:38:14: worker thread finished; awaiting finish of 12 more threads
INFO - 23:38:14: worker thread finished; awaiting finish of 11 more threads
INFO - 23:38:14: worker thread finished; awaiting finish of 10 more threads
INFO - 23:38:14: worker thread finished; awaiting finish of 9 more threads
INFO - 23:38:14: worker thread finished; awaiting finish of 8 more threads
INFO - 23:38:15: worker thread finished; awaiting finish of 7 more threads
INFO - 23:38:15: worker thread finished; awaiting finish of 6 more threads
INFO - 23:38:15: worker thread finished; awaiting finish of 5 more threads
INFO - 23:38:15: worker thread finished; awaiti

INFO - 23:38:34: worker thread finished; awaiting finish of 10 more threads
INFO - 23:38:34: worker thread finished; awaiting finish of 9 more threads
INFO - 23:38:34: worker thread finished; awaiting finish of 8 more threads
INFO - 23:38:34: worker thread finished; awaiting finish of 7 more threads
INFO - 23:38:34: worker thread finished; awaiting finish of 6 more threads
INFO - 23:38:34: worker thread finished; awaiting finish of 5 more threads
INFO - 23:38:34: worker thread finished; awaiting finish of 4 more threads
INFO - 23:38:34: worker thread finished; awaiting finish of 3 more threads
INFO - 23:38:34: worker thread finished; awaiting finish of 2 more threads
INFO - 23:38:34: worker thread finished; awaiting finish of 1 more threads
INFO - 23:38:34: worker thread finished; awaiting finish of 0 more threads
INFO - 23:38:34: EPOCH - 16 : training on 2893505 raw words (2703308 effective words) took 3.9s, 693566 effective words/s
INFO - 23:38:35: EPOCH 17 - PROGRESS: at 21.39% exam

INFO - 23:38:54: worker thread finished; awaiting finish of 4 more threads
INFO - 23:38:54: worker thread finished; awaiting finish of 3 more threads
INFO - 23:38:54: worker thread finished; awaiting finish of 2 more threads
INFO - 23:38:54: worker thread finished; awaiting finish of 1 more threads
INFO - 23:38:54: worker thread finished; awaiting finish of 0 more threads
INFO - 23:38:54: EPOCH - 21 : training on 2893505 raw words (2703606 effective words) took 3.8s, 711971 effective words/s
INFO - 23:38:55: EPOCH 22 - PROGRESS: at 22.05% examples, 594482 words/s, in_qsize 26, out_qsize 3
INFO - 23:38:56: EPOCH 22 - PROGRESS: at 49.95% examples, 667085 words/s, in_qsize 29, out_qsize 0
INFO - 23:38:57: EPOCH 22 - PROGRESS: at 77.44% examples, 685436 words/s, in_qsize 29, out_qsize 0
INFO - 23:38:58: worker thread finished; awaiting finish of 14 more threads
INFO - 23:38:58: worker thread finished; awaiting finish of 13 more threads
INFO - 23:38:58: worker thread finished; awaiting fini

INFO - 23:39:14: EPOCH 27 - PROGRESS: at 23.77% examples, 632220 words/s, in_qsize 26, out_qsize 3
INFO - 23:39:15: EPOCH 27 - PROGRESS: at 50.93% examples, 686046 words/s, in_qsize 29, out_qsize 0
INFO - 23:39:16: EPOCH 27 - PROGRESS: at 78.10% examples, 697159 words/s, in_qsize 30, out_qsize 0
INFO - 23:39:17: worker thread finished; awaiting finish of 14 more threads
INFO - 23:39:17: worker thread finished; awaiting finish of 13 more threads
INFO - 23:39:17: worker thread finished; awaiting finish of 12 more threads
INFO - 23:39:17: worker thread finished; awaiting finish of 11 more threads
INFO - 23:39:17: worker thread finished; awaiting finish of 10 more threads
INFO - 23:39:17: worker thread finished; awaiting finish of 9 more threads
INFO - 23:39:17: worker thread finished; awaiting finish of 8 more threads
INFO - 23:39:17: worker thread finished; awaiting finish of 7 more threads
INFO - 23:39:17: worker thread finished; awaiting finish of 6 more threads
INFO - 23:39:17: worker

Time to train the model: 1.99 mins


In [12]:
w2v_model.init_sims(replace=True)

INFO - 23:39:38: precomputing L2-norms of word weight vectors


In [17]:
# K product_id daného v závorce jako string najde nejvhodnější jiné produkty:
w2v_model.wv.most_similar(positive=["1709503"])

[('2463628', 0.7064393162727356),
 ('1709501', 0.6800525188446045),
 ('877830', 0.6790813207626343),
 ('877783', 0.6780321598052979),
 ('2463636', 0.6767614483833313),
 ('1336396', 0.6749566793441772),
 ('2463773', 0.6723500490188599),
 ('1709499', 0.6696857810020447),
 ('509505', 0.6642148494720459),
 ('1709495', 0.6592025756835938)]

In [14]:
#Vypsání produktů, které jsou nejvícekrát obsazené ve vzorku tabulky product_views_cz:
temp = data_2020.groupby('product_id').apply(len)
print(temp[temp >= 200].sort_values())

product_id
888983           200
584830           200
891168           200
956091           200
1709503          200
                ... 
2401410         1186
100016289008    1203
848128007       1243
1264679001      1277
100024602647    2372
Length: 810, dtype: int64


In [15]:
#Ulozi model do souboru
w2v_model.save("w2v.model")

INFO - 23:40:31: saving Word2Vec object under w2v.model, separately None
INFO - 23:40:31: storing np array 'vectors' to w2v.model.wv.vectors.npy
INFO - 23:40:32: not storing attribute vectors_norm
INFO - 23:40:32: storing np array 'syn1neg' to w2v.model.trainables.syn1neg.npy
INFO - 23:40:33: not storing attribute cum_table
INFO - 23:40:33: saved w2v.model
